In [5]:
import whisper
from pprint import pprint
import torch

modelName = "medium"
# Load the model
# model = whisper.load_model("medium")
model = whisper.load_model(modelName)

print(f"CUDA={torch.cuda.is_available()}")


In [6]:
# Now I need to scan a folder and find all the files that contains files with trec extensions
import os
import re

# Define the folder to scan
folder = "S:\\OneDrive\\Documenti\\ordine_ingegneri\\Forensics"

# Define the regex to find the files
pattern = re.compile(r".*\.trec$")
files = []
for root, dirs, filenames in os.walk(folder):
    for filename in filenames:
        if pattern.match(filename):
            files.append(os.path.join(root, filename))

# Dump all file names
pprint(files)

In [7]:
# Now Use ffmpeg to extract audio from these files. Iterate all the files and generate a file name with wav extension
import subprocess
audiofiles = []
for file in files:
    # Generate the output file name
    output_file = file.replace(".trec", ".wav")

    # If a previous file exists delete it
    if os.path.exists(output_file):
        os.remove(output_file)
        
    # Execute the command
    command = f"ffmpeg -i {file} -vn -acodec pcm_s16le -ar 44100 -ac 2 {output_file}"
    print(command)
    subprocess.run(command, shell=True)
    audiofiles.append(output_file)

In [8]:
output_file = folder + f"/{modelName}_output.txt"
# if file output extsts delete
if os.path.exists(output_file):
    os.remove(output_file)

with open(output_file, "w") as f:

    for file in audiofiles:
        print(f"Processing file {file}")

        single_segment_output_file = file.replace(".wav", f".{modelName}.txt")
        transcription_string = ""

        # if output file exists load content into transcription_string
        if os.path.exists(single_segment_output_file):
            print (f"File {single_segment_output_file} exists. Loading content")
            with open(single_segment_output_file, "r") as f2:
                transcription_string = f2.read()
        else:
            # Transcribe the file
            print (f"Trnscribe file {file} with whisper")
            transcription_options = {
                "verbose" : False,
                "task": "transcribe"
            }
            transcription = model.transcribe(file, **transcription_options)
            
            for segment in transcription['segments']:
                text = segment['text']
                transcription_string += text + "\n"

            # write also in a transcription file with the same name of the wav file but extension txt
            with open(single_segment_output_file, "w") as f2:
                f2.write(transcription_string)

        # Write the transcription to the file
        f.write(f"section {file}\n")
        f.write(transcription_string)